## Крупные библиотеки NLP: SpaCy, Stanza (StanfordNLP), Natasha

#### SpaCy

Спейси - современная библиотека, которая написана в Cython и использует нейронные сети. Интерфейс спейси довольно удобный и однообразный. Центральное понятие для спейси - это pipeline: то есть, набор действий, которые спейси совершает с текстом. Чтобы обработать текст на любом из языков, представленных в библиотеке ([список](https://spacy.io/usage/models)), достаточно завести пустой пайплайн:

In [1]:
import spacy

nlp = spacy.blank('en')
doc = nlp('My beautiful sentence is here.')

Предупреждения может выдавать библиотека tensorflow, которая сообщает, что у вас не настроена CUDA, но их можно игнорировать, как и предлагается. 

Пустой пайплайн превращает наш текст в особый объект, в котором текст разделен на токены, и можно у этих токенов смотреть самые простые характеристики:

In [2]:
for token in doc:
    print(f'№ {token.i}. Токен: {token.text:>15}. Является словом: {token.is_alpha}. Является пунктуацией: {token.is_punct}. Похож на число: {token.like_num}')

№ 0. Токен:              My. Является словом: True. Является пунктуацией: False. Похож на число: False
№ 1. Токен:       beautiful. Является словом: True. Является пунктуацией: False. Похож на число: False
№ 2. Токен:        sentence. Является словом: True. Является пунктуацией: False. Похож на число: False
№ 3. Токен:              is. Является словом: True. Является пунктуацией: False. Похож на число: False
№ 4. Токен:            here. Является словом: True. Является пунктуацией: False. Похож на число: False
№ 5. Токен:               .. Является словом: False. Является пунктуацией: True. Похож на число: False


Объект doc также позволяет смотреть спаны (несколько токенов, срез текста):

In [3]:
span = doc[1:3]
span.text

'beautiful sentence'

Ну, это все прекрасно, но хотелось бы чего-то большего. Для spacy есть довольно много предобученных моделей для разных языков (список см. выше). Модельки нужно устанавливать (скачивать) с помощью команды в командной строке - она написана у них на сайте (python -m spacy download имя_модели). Когда вы загрузили модельку, вы можете ее использовать в коде:

In [4]:
nlp = spacy.load('en_core_web_lg')

Теперь уже можно получить сведения поинтереснее:

In [5]:
doc = nlp('The 4th Army was a Royal Yugoslav Army formation mobilised prior to the German-led invasion of the Kingdom of Yugoslavia during World War II.')

In [14]:
for token in doc:
    print(f'{token.i:2}. {token.text:15} POS: {token.pos_:6} SyntR: {token.dep_:9} Head: {token.head.text}')

 0. The             POS: DET    SyntR: det       Head: Army
 1. 4th             POS: ADJ    SyntR: amod      Head: Army
 2. Army            POS: PROPN  SyntR: nsubj     Head: was
 3. was             POS: AUX    SyntR: ROOT      Head: was
 4. a               POS: DET    SyntR: det       Head: formation
 5. Royal           POS: PROPN  SyntR: compound  Head: Army
 6. Yugoslav        POS: ADJ    SyntR: amod      Head: Army
 7. Army            POS: PROPN  SyntR: compound  Head: formation
 8. formation       POS: NOUN   SyntR: attr      Head: was
 9. mobilised       POS: VERB   SyntR: acl       Head: formation
10. prior           POS: ADV    SyntR: advmod    Head: mobilised
11. to              POS: ADP    SyntR: prep      Head: prior
12. the             POS: DET    SyntR: det       Head: invasion
13. German          POS: PROPN  SyntR: npadvmod  Head: led
14. -               POS: PUNCT  SyntR: punct     Head: led
15. led             POS: VERB   SyntR: amod      Head: invasion
16. invasion    

Грамматические характеристики можно тоже посмотреть:

In [15]:
for token in doc[:3]:
    print(token.morph)

Definite=Def|PronType=Art
Degree=Pos
Number=Sing


Также spacy позволяет разметить именованные сущности и посмотреть, что получилось:

In [7]:
for ent in doc.ents:
    print(f'Entity: {ent.text:30} Label: {ent.label_}')

Entity: The 4th Army                   Label: ORG
Entity: Royal Yugoslav Army            Label: ORG
Entity: German                         Label: NORP
Entity: the Kingdom of Yugoslavia      Label: GPE
Entity: World War II                   Label: EVENT


Если аббревиатуры вас смущают, spacy легко предоставит расшифровки (для всех!):

In [33]:
spacy.explain('NORP')

'Nationalities or religious or political groups'

In [34]:
spacy.explain('pobj')

'object of preposition'

Итак, а теперь самое интересное - это обертки для spacy + udpipe. UD, как известно - очень известный и влиятельный проект (чешский), цель которого - унифицировать тагсет для всех языков мира, причем как в морфологии, так и в синтаксисе. Неудивительно, что, поскольку в UD существует большое количество размеченных датасетов (русским языком активно занималась О. Ляшевская), то они обучили и свой парсер. Сам парсер написан в плюсах и не очень дружелюбен, но обертки для spacy делают жизнь проще. 

Внимание: я обычно это забываю, но прежде чем установить udpipe, **нужно поставить C++ Build Tools** (с сайта майкрософт, это бесплатно). Это нужно, чтобы скомпилировать udpipe из исходников в сях. Адрес сайта, откуда брать их, сам pip обычно подсказывает, но в целом можно просто погуглить. 

Две оболочки для udpipe, которые мы посмотрели - это spacy_udpipe и corpy. Обе нужно устанавливать:

    pip install spacy_udpipe
    pip install corpy
    
Для spacy_udpipe вообще ничего больше не нужно, там максимально автоматизированно скачиваются модельки. Для corpy приходится скачивать искомую модель руками, [отсюда](https://lindat.mff.cuni.cz/repository/xmlui/handle/11234/1-3131). Эту модель вы можете положить куда угодно, главное потом указать corpy путь к ней. 

In [35]:
import spacy_udpipe

spacy_udpipe.download('en')  # эту команду достаточно выполнить только один раз - она как nltk.download()

Already downloaded a model for the 'en' language


In [37]:
nlp = spacy_udpipe.load('en')
doc = nlp('The 4th Army was a Royal Yugoslav Army formation mobilised prior to the German-led invasion of the Kingdom of Yugoslavia during World War II.')
for token in doc:
    print(f'{token.i:2}. {token.text:15} Lemma: {token.lemma_:15} POS: {token.pos_:6} SyntR: {token.dep_:9} Head: {token.head.text}')

 0. The             Lemma: the             POS: DET    SyntR: det       Head: Army
 1. 4th             Lemma: 4th             POS: ADJ    SyntR: amod      Head: Army
 2. Army            Lemma: Army            POS: PROPN  SyntR: nsubj     Head: formation
 3. was             Lemma: be              POS: AUX    SyntR: cop       Head: formation
 4. a               Lemma: a               POS: DET    SyntR: det       Head: formation
 5. Royal           Lemma: Royal           POS: PROPN  SyntR: compound  Head: Army
 6. Yugoslav        Lemma: Yugoslav        POS: PROPN  SyntR: compound  Head: Army
 7. Army            Lemma: Army            POS: PROPN  SyntR: compound  Head: formation
 8. formation       Lemma: formation       POS: NOUN   SyntR: nsubj     Head: mobilised
 9. mobilised       Lemma: mobilise        POS: VERB   SyntR: ROOT      Head: mobilised
10. prior           Lemma: prior           POS: ADJ    SyntR: case      Head: invasion
11. to              Lemma: to              POS: ADP  

То же самое можно сделать в corpy:

In [38]:
from corpy.udpipe import Model

model = Model('english-partut-ud-2.5-191206.udpipe')  
# тут и нужно указать путь к вашей модели. Если она лежит в той же папке, что и скрипт, достаточно только имени файла

sents = model.process('The 4th Army was a Royal Yugoslav Army formation mobilised prior to the German-led invasion of the Kingdom of Yugoslavia during World War II.')

corpy возвращает генератор (то есть, итерируемый объект, который как магазин автомата, расстреляли все патроны - он опустел; повторно по генератору итерироваться нельзя). Генератор на каждом шаге выдает предложение (объект специального класса Sentence()), а в предложении - объекты класса Word(). 

In [40]:
for sent in sents:
    for word in sent.words:
        print(f'{word.form:15} Лемма: {word.lemma:15} POS: {word.upostag} Грам. инфа: {word.feats}')
print('Алилуя!')

<root>          Лемма: <root>          POS: <root> Грам. инфа: <root>
The             Лемма: the             POS: DET Грам. инфа: Definite=Def|PronType=Art
4th             Лемма: 4th             POS: ADJ Грам. инфа: Degree=Pos
Army            Лемма: army            POS: NOUN Грам. инфа: Number=Sing
was             Лемма: be              POS: AUX Грам. инфа: Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin
a               Лемма: a               POS: DET Грам. инфа: Definite=Ind|Number=Sing|PronType=Art
Royal           Лемма: royal           POS: PROPN Грам. инфа: 
Yugoslav        Лемма: Yugoslav        POS: PROPN Грам. инфа: 
Army            Лемма: army            POS: PROPN Грам. инфа: 
formation       Лемма: formation       POS: NOUN Грам. инфа: Number=Sing
mobilised       Лемма: mobilize        POS: VERB Грам. инфа: Mood=Ind|Person=3|Tense=Past|VerbForm=Fin
prior           Лемма: prior           POS: ADJ Грам. инфа: Degree=Pos
to              Лемма: to              POS: ADP Грам

у corpy, кстати, есть свой способ вывода имеющейся информации (хотя, по-моему, в юпитере и так красиво выводится...)

In [43]:
from corpy.udpipe import pprint

pprint(list(model.process('The 4th Army was a Royal Yugoslav Army formation mobilised prior to the German-led invasion of the Kingdom of Yugoslavia during World War II.')))

[Sentence(
   comments=[
     '# newdoc',
     '# newpar',
     '# sent_id = 1',
     '# text = The 4th Army was a Royal Yugoslav Army formation mobilised prior to the German-led invasion of the Kingdom of Yugoslavia during World War II.'],
   words=[
     Word(id=0, <root>),
     Word(id=1,
          form='The',
          lemma='the',
          xpostag='RD',
          upostag='DET',
          feats='Definite=Def|PronType=Art',
          head=3,
          deprel='det'),
     Word(id=2,
          form='4th',
          lemma='4th',
          xpostag='A',
          upostag='ADJ',
          feats='Degree=Pos',
          head=3,
          deprel='amod'),
     Word(id=3,
          form='Army',
          lemma='army',
          xpostag='S',
          upostag='NOUN',
          feats='Number=Sing',
          head=9,
          deprel='nsubj'),
     Word(id=4,
          form='was',
          lemma='be',
          xpostag='V',
          upostag='AUX',
          feats='Mood=Ind|Number=Sing|Person=3

### Stanza

In [ ]:
!pip install stanza

Загрузка моделей

In [9]:
import stanza

In [ ]:
nlp_ru = stanza.Pipeline(lang='ru')
nlp_en = stanza.Pipeline(lang='en', processors='tokenize, pos, constituency')
nlp_fr = stanza.Pipeline(lang='fr', processors='tokenize, mwt')

Токенизация, сегментация по предложениям

In [21]:
text = '''Архитектура Византии — совокупность традиций строительства и архитектуры в поздней Римской империи и в Византии в период с начала IV века по середину XV века. В качестве отдельных направлений исследования выделяют религиозную архитектуру Византии, византийскую фортификацию и гражданское строительство, включающее дворцы, общественные сооружения и частные дома. Также в рамках данной дисциплины изучают традиции строительного ремесла и декоративного искусства.'''

doc = nlp_ru(text)

In [17]:
print(*[sentence.text for sentence in doc.sentences], sep='\n')

Архитектура Византии — совокупность традиций строительства и архитектуры в поздней Римской империи и в Византии в период с начала IV века по середину XV века.
В качестве отдельных направлений исследования выделяют религиозную архитектуру Византии, византийскую фортификацию и гражданское строительство, включающее дворцы, общественные сооружения и частные дома.
Также в рамках данной дисциплины изучают традиции строительного ремесла и декоративного искусства.


In [ ]:
for i, sentence in enumerate(doc.sentences):
  print(f'====== Sentence {i + 1} tokens =======')
  print(*[f'id: {token.id}\ttext: {token.text}' for token in sentence.tokens], sep='\n')

In [ ]:
text_fr = '''Il est révélé par les romans Extension du domaine de la lutte (1994) et, surtout, Les Particules élémentaires (1998), qui le fait connaître d'un large public.'''

doc_fr = nlp_fr(text_fr)
for token in doc_fr.sentences[0].tokens:
    print(f'token: {token.text}\twords: {", ".join([word.text for word in token.words])}')

Лемматизация

In [ ]:
print(*[f'word: {word.text+" "}\tlemma: {word.lemma}' for sent in doc.sentences for word in sent.words], sep='\n')

Морфопарсинг

In [ ]:
print(*[f'word: {word.text}\tupos: {word.upos}\txpos: {word.xpos}\tfeats: {word.feats if word.feats else "_"}' for sent in doc.sentences for word in sent.words], sep='\n')

Парсинг синтаксических зависимостей

In [ ]:
print(*[f'id: {word.id}\tword: {word.text}\thead id: {word.head}\thead: {sent.words[word.head-1].text if word.head > 0 else "root"}\tdeprel: {word.deprel}' for sent in doc.sentences for word in sent.words], sep='\n')

In [ ]:
doc.sentences[0].print_dependencies()

Парсинг составляющих (для русского недоступен)

In [ ]:
doc_en = nlp_en('This is a sentence for parsing constituencies.')

for sentence in doc_en.sentences:
    print(sentence.constituency)

### natasha

In [ ]:
!pip install natasha

Морфосинтаксический парсинг

In [2]:
from natasha import (
    Segmenter,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    
    Doc
)

In [ ]:
segmenter = Segmenter()  # токенизация и разделение на предложения
emb = NewsEmbedding()  # эмбеддинги
morph_tagger = NewsMorphTagger(emb)  # морфология
syntax_parser = NewsSyntaxParser(emb) # синтаксис

text = '29 марта 2017 года правительство Великобритании инициировало процедуру выхода в соответствии со статьёй 50 Договора о Европейском союзе; первоначально планировалось, что Великобритания покинет Европейский союз через два года, 29 марта 2019 года в 23:00 по Гринвичу.'
doc = Doc(text)

doc.segment(segmenter)
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)
sent = doc.sents[0]
sent.morph.print()
sent.syntax.print()

Распознание именованных сущностей

In [ ]:
from natasha import NewsNERTagger

ner_tagger = NewsNERTagger(emb)
doc.tag_ner(ner_tagger)
doc.ner.print()

Лемматизация

In [ ]:
from natasha import MorphVocab

morph_vocab = MorphVocab()

for token in doc.tokens:
  token.lemmatize(morph_vocab)
  print(token.lemma)

Нормализация именованных сущностей

In [ ]:
for span in doc.spans:
    span.normalize(morph_vocab)
   
{_.text: _.normal for _ in doc.spans}